In [ ]:
%%capture

!git clone https://github.com/agoryuno/deepsolo-onnx

import os
import shutil
os.chdir('deepsolo-onnx')

!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt

!python setup.py sdist bdist_wheel
!pip install dist/*.whl

shutil.copy('configs/Base_det_export.yaml', '/content')

os.chdir('/content')

!rm -rf deepsolo-onnx

In [ ]:
# This is for quick reloads during development- DON"T EXECUTE
#
assert False
!pip install --force-reinstall --no-deps git+https://github.com/agoryuno/deepsolo-onnx

In [ ]:
# Specify the path to the DeepSolo checkpoint you want to export
# you can download one from the main DeepSolo repo: 
# https://github.com/ViTAE-Transformer/DeepSolo
#
CHECKPOINT = "/content/drive/MyDrive/deepsolo/vitaev2-s_pretrain_synth-tt-mlt-13-15-textocr.pth"

# This is the path for the output
#
OUTPATH = "deepsolo.onnx"

# Input and output image dimensions - these are fixed 
# for the exported ONNX model, so choose wisely
# greater dims => greater checkpoint size
DIMS = (512,512)

from DeepSolo.onnx_model import SimpleONNXReadyModel

# This is the default config file
# you probably should leave it alone
#
CONFIG = "/content/Base_det_export.yaml"

# The number of channels in an image
# This can be left at default
#
CHANNELS = 3

model = SimpleONNXReadyModel(CONFIG, CHECKPOINT)

In [3]:
# Creating a fake "image" to set the input-output dimensions
# of the exported model's data
#

import numpy as np

img = np.random.randint(0, 255, (CHANNELS, *DIMS))

In [ ]:
# Export the checkpoint
#
import torch.onnx
import torch

img_t = torch.from_numpy(img)

torch.onnx.export(model.model,
           [img_t],
           OUTPATH,
           export_params=True,
           do_constant_folding=True,)